# Lab 4 - praca z atrybutami nienumerycznymi oraz skalowanie i transformacja wartości

## Dane tekstowe

W zbiorach danych często pojawiają się dane tekstowe mające charakter kategorialny. Z uwagi na fakt, że algorytmy uczenia maszynowego operują tylko i wyłącznie na wartościach numerycznych, dane tekstowe wymagają przetworzenia do postaci liczbowej. W takiej sytuacji z pomocą przychodzi klasa *OrdinalEncoder* zawarta w module *preprocessing* biblioteki *Scikit-learn*.

In [ ]:
from sklearn.datasets import fetch_kddcup99

data = fetch_kddcup99(as_frame=True)['frame']

In [ ]:
data

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
labels = encoder.fit_transform(data[['protocol_type']])

Interfejs biblioteki *Scikit-learn* umożliwia alternatywne wywoływanie metod *fit* oraz *transform*, które występują po sobie w postaci *fit_transform*.

In [ ]:
labels

Uzyskane wartości numeryczne przypisane poszczególnym wartościom tekstowym znalazły się w obiekcie *labels*. Uzyskane wartości można z powodzeniem wykorzystać w ramce danych. Klucz, według którego wartości tekstowe były transformowane do wartości numerycznych znajduje się w atrybucie *categories_* obiektu *encoder*, gdzie pozycja tablicy odpowiada przypisanej liczbie.

In [ ]:
encoder.categories_

Algorytmy uczące się posiadają zdolność do rozpoznawania wzorców definiowalnych przez liczby. W związku z tym niektóre z nich mogą bardziej istotnie traktować wartość liczbową przypisaną protokołowi UDP niż protokołowi ICMP. W związku z tym warto rozważyć zastosowanie tzw. **kodowanie gorącojedynkowe**, które polega na utworzeniu wektora rzadkiego o rozmiarze *n* (n = liczba różnych wartości tekstowych) z jedną jedynką na pozycji wskazującej daną kategorię. W tym celu należy wykorzystać klasę *OneHotEncoder*.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
protocol_onehot = encoder.fit_transform(data[['protocol_type']])

Wynikiem jest obiekt zawierający macierz rzadką, co niesie korzyść w postaci oszczędności pamięci. Za pomocą metody *toarray* można jednak dokonać transformacji do postaci tablicy numpy.

In [ ]:
protocol_onehot

In [ ]:
protocol_onehot.toarray()

Zastąpienie oryginalnego atrybutu wersją gorącojedynkową niesie za sobą zwiększenie ogólnej liczby atrybutów w ramce. Na każdą kategorię powstaje w takiej sytuacji jeden atrybut.

In [ ]:
import pandas as pd

onehot_df = pd.DataFrame(
    protocol_onehot.toarray(),
    columns=encoder.get_feature_names_out(),
    index=data.index
    )

In [ ]:
data = data.join(onehot_df)

In [ ]:
data

## Skalowanie i transformacja atrybutów liczbowych

Wiele algorytmów uczących się źle radzi sobie podczas pracy z atrybutami numerycznymi, których wartości mieszczą się w różnych skalach (np. 0-1 oraz 4-290). Istnieją jednak metody sprawnie umożliwiające transformację wartości w atrybutach do ujednoliconego zakresu.

Jednym z najpopularniejszych podejść do skalowania atrybutów liczbowych jest normalizacja (skalowanie min-max), które polega na umieszczeniu wszystkich wartości w wyznaczonym zakresie (np. 0-1): $x' = \frac{x - min(x)}{max(x) - min(x)}$

W bibliotece *Scikit-learn* do normalizacji służy klasa *MinMaxScaler* zawarta w module *preprocessing*, gdzie można wskazać oczekiwany zakres.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
src_bytes_scaled = scaler.fit_transform(data[['src_bytes']])

In [ ]:
src_bytes_scaled

In [ ]:
src_bytes_scaled.max(), src_bytes_scaled.min()

Standaryzacja to proces polegający na wyśrodkowaniu danych oraz zachowaniu wskazanych parametrów dotyczących rozrzutu wartości: $x' = \frac{x - u}{s}$, gdzie $u$ - średnia arytmetyczna, $s$ - odchylenie standardowe. W bibliotece *Scikit-learn* do standaryzacji służy klasa *StandardScaler* zawarta w module *preprocessing*.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dst_bytes_scaled = scaler.fit_transform(data[['dst_bytes']])

In [ ]:
dst_bytes_scaled

Gdzie i kiedy stosować normalizację i standaryzację? Zwykle metody dopasowywane są eksperymentalnie. Warto mieć jednak na uwadze, że stadaryzacja jest czuła na wartości odstające, co może wpływać na różnice w zakresie wartości atrybutów.

## Symetria wartości numerycznych

In [ ]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)['frame']

In [ ]:
data

W pewnych przypadkach histogramy atrybutów numerycznych potrafią być asymetryczne.

In [ ]:
data['MedInc'].hist(bins=100)

Powyższy histogram jest prawoskośny, co oznacza że jego asymetria jest spowodowana nasileniem występowania wartości występujących po lewej stronie histogramu. Analogicznie wygląda sytuacja w przypadku lewoskośności. Rozwiązaniem tego problemu może być zastosowanie skali logarytmicznej lub pierwiastkowej.

In [ ]:
import numpy as np

In [ ]:
np.sqrt(data['MedInc']).hist(bins=100)

In [ ]:
np.log(data['MedInc']).hist(bins=100)

## Zadania

1. Pobrać i wczytać zbiór danych danych dostępny pod adresem: https://archive.ics.uci.edu/dataset/10/automobile.

2. Usunąć wiersze ze zbioru danych, w których występują wartości wybrakowane.

3. Dokonać zamiany wartości tekstowych na numeryczne przy użyciu dwóch poznanych metod, usuwając oryginalny atrybut.

4. Dla atrybutów numerycznych, których wartości prezentują lewo- lub prawoskośność, zastosować poznane metody poprawy symetrii. Można zastosować dowolne rozwiązania (nie tylko logarytm i pierwiastek), do uzyskania finalnego rozwiązania warto zastosować jedno rozwiązanie, które zapewnia najbardziej zauważalny efekt na histogramie wynikowym.

5. Dokonać normalizacji lub standaryzacji atrybutów numerycznych. Wartości w atrybutach numerycznych powinny w tym celu zostać zamienione na nowe (przetransformowane).